In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
import google.generativeai as genai
import time

load_dotenv("../.env",override=True)
GOOGLE_API_KEY = getenv("GEMINI_API_KEY")
OPENAI_API_KEY = getenv("OPENAI_API_KEY")






In [ ]:
class chatGPT():
    def __init__(self):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        
    def ask(self, q):
        stream = self.client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": q}],
            stream=True,
            temperature=0
        )
        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response += chunk.choices[0].delta.content

        self.response = response
        return response

    def get_embedding(self,text, model="text-embedding-3-large"):
       text = text.replace("\n", " ")
       return self.client.embeddings.create(input = [text], model=model).data[0].embedding

x_chat = chatGPT()


In [ ]:
df_exp = pd.read_csv("./input/gpts_labels_new.csv")
df_exp = df_exp[(df_exp.human_labels!='E0') | (df_exp.gpt_4_exposure!='E0')]
df_exp["description"] = df_exp.apply(lambda x: f"{x.Title.capitalize()} that {x.Task.lower()}",axis=1)
df_exp[(df_exp.gpt_4_exposure=='EO')]
df_exp.gpt_4_exposure.unique()


for i, row in df_exp[df_exp.Title == "Sales Managers"].iterrows():
    print(row.description)
    
results = []
for i, x in df_exp.reset_index().iterrows():
    description = x["description"]
    print(f"Processing {str(i), len(results)}: {description}")
    result = x_chat.get_embedding(description)
    results.append([description,result])

df = pd.DataFrame(results, columns=['Description', 'Embeddings'])
df_exp = pd.concat([df_exp.reset_index(), df], axis=1)


# df_exp.to_csv("gpt_exposure_embeddings.csv")